In [92]:
# Import the yfinance. If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf
import pandas as pd
import numpy as np

In [93]:
# get daily, yearly or hourly finance data
stock_data = yf.download(tickers="SPY", period="1d", interval="1m")

[*********************100%***********************]  1 of 1 completed


In [98]:
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-05-29 09:30:00-04:00,525.679993,525.989990,525.599976,525.979980,525.979980,1502384
2024-05-29 09:31:00-04:00,525.979980,526.020020,525.789978,525.789978,525.789978,189565
2024-05-29 09:32:00-04:00,525.789978,525.849976,525.630005,525.780029,525.780029,155171
2024-05-29 09:33:00-04:00,525.789978,525.815002,525.544006,525.549988,525.549988,170719
2024-05-29 09:34:00-04:00,525.559998,525.669983,525.460022,525.614990,525.614990,195815


In [95]:
# data.groupby(pd.Grouper(freq='15Min')).size()
df = stock_data.reset_index().groupby(
    pd.Grouper(freq='15Min', key="Datetime")
).agg(
    open=("Open", "first"), 
    close=("Close", "last"),
    low=("Low", "min"),
    high=("High", "max"),
)

In [96]:
df["return"] = df["close"] - df["open"]
df.reset_index().head()


,Datetime,open,close,low,high,return
0,2024-05-29 09:30:00-04:00,525.679993,525.515015,525.424988,526.020020,-0.164978
1,2024-05-29 09:45:00-04:00,525.510010,526.200012,525.369995,526.419983,0.690002
2,2024-05-29 10:00:00-04:00,526.205017,526.400024,526.010010,526.543701,0.195007
3,2024-05-29 10:15:00-04:00,526.424988,526.729980,525.859985,526.969971,0.304993
4,2024-05-29 10:30:00-04:00,526.755981,526.929993,526.755981,527.304993,0.174011


In [97]:
# let's plot
import altair as alt
from vega_datasets import data

# source = data.ohlc()
# display(source.head())
source = df.reset_index()

open_close_color = alt.condition(
    "datum.open <= datum.close",
    alt.value("#06982d"),
    alt.value("#ae1325")
)

base = alt.Chart(source).encode(
    alt.X('Datetime:T')
        .axis(format='%H:%M', labelAngle=-80)
        .title('date-time'),
    color=open_close_color
)

rule = base.mark_rule(size=1).encode(
    alt.Y('low:Q')
        .title('Price')
        .scale(zero=False),
    alt.Y2('high:Q')
)

bar = base.mark_bar(size=10).encode(
    alt.Y('open:Q'),
    alt.Y2('close:Q')
)

(rule+bar).properties(
    width=800,
    height=300
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
)

alt.LayerChart(...)